In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')

"""
looking through all valid spacings (if it si invalid it goes below 0)
and displaying minimal maximal and rounded mean spacing and median
in my case median and mean values are close - and using the median values will lead to a bit less interpolations later
"""
spacingDict={}
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
    for addedKey in ['_spac_x','_spac_y','_spac_z']:   
        colName = keyWord+addedKey
        liist = list(filter(lambda it: it>0 ,df[colName].to_numpy() ))
        minn=np.min(liist)                
        maxx=np.max(liist)
        meanRounded = round((minn+maxx)/2,1)
        medianRounded = round(np.median(liist),1)
        spacingDict[colName]=(minn,maxx,meanRounded,medianRounded)

spacingDict



{'t2w_spac_x': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_y': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_z': (2.200000060372773, 5.000000066297424, 3.6, 3.0),
 'adc_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'cor_spac_x': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_y': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_z': (0.9999999999999636, 4.49999998188102, 2.7, 3.0),
 'hbv_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'sag_spac_x': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_y': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_z': (2.999999882111563, 4.000404550206146, 3.5, 3.6)}

In [3]:
from KevinSR import mask_interpolation, SOUP_GAN
import os
import numpy as np
from scipy import ndimage, interpolate
from scipy.ndimage import zoom
from KevinSR import SOUP_GAN
import monai
import SimpleITK as sitk
from copy import deepcopy


def superSampleinZ(data, currentSpacZ, targetSpacZ):
    """
    perform supersampling in Z direction
    currentSpac, targetSpac - current and target z spacing - float numbers
    """
    Z_FAC = currentSpacZ/targetSpacZ
    # Call the SR interpolation tool from KevinSR
    thins_gen = SOUP_GAN(data, Z_FAC, 0)
    return thins_gen
    
def subsample(image,orig_spacing, targetSpacZ):
    """
    perform subsampling to targetSpac
    TODO() bicubic interpolation adapted from 
        https://itk.org/Doxygen/html/WikiExamples_2ImageProcessing_2Upsampling_8cxx-example.html
        https://discourse.itk.org/t/resample-produces-blurry-results-when-just-cropping/4473/8
    """ 
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(targetSpacZ)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())
    resample.SetInterpolator(sitk.sitkBSpline)
    
    new_size = tuple([int(image.GetSize()[0]*(orig_spacing[0]/targetSpacZ[0])),
                int(image.GetSize()[1]*(orig_spacing[1]/targetSpacZ[1])),
                int(image.GetSize()[2]*(orig_spacing[2]/targetSpacZ[2]) )  ]  )
    
    print(f"new_size {new_size}")
    resample.SetSize(new_size)

    res = resample.Execute(image)
    print(f"subsampled {res.GetSize()}")
    return res 


def transformSpacingTo(path, targetSpac,finalSize):
    """
    transforming spacing of the image which path is supplied to target spacing values
    upsampling will be performed with SOUP-GAN  and downsampling will be done via simple itk bicubic interpolation
    """
    image = sitk.ReadImage(path)
    spac = image.GetSpacing()
    origSpac=spac
    spacGoal=list(deepcopy(spac))    
    
    for axis in [0,1,2]:
        #tuple are immutable ...
        spacGoal[axis]=targetSpac[axis]
# abs        print(f"spac {spac} axis {axis} spacing goal {spacGoal} ")
#         if(targetSpac[axis]>spac[axis]):
#             #if target spacing is bigger we need to subsample

#             #overwriting image by resampled one
#             image=subsample(image, tuple(spacGoal))
#             pass
        if(targetSpac[axis]<spac[axis]):
            #we need to perform supersampling - now as supersampling can be performed only in z direction
            #we need to rotate image accordingly       
            data = sitk.GetArrayFromImage(image)
            if(axis==2):
                transformed = np.moveaxis(data, 0, 2)
                data=superSampleinZ(transformed, spac[axis], targetSpac[axis])
                #getting back original shape
                data = np.moveaxis(data, 2, 0)
            if(axis==1):
                transformed = np.moveaxis(data, 1, 2)
                data=superSampleinZ(transformed, spac[axis], targetSpac[axis])
                #getting back original shape
                data = np.moveaxis(data, 2, 1)                
            if(axis==3):
                # no need to transform
                data=superSampleinZ(data, spac[axis], spac[axis], targetSpac[axis])
            #recreate the image and ovewrite original
            print(f"data shape {data.shape}")
            image1 = sitk.GetImageFromArray(data)
            image1.SetSpacing(tuple(spacGoal)) #updating spacing
            image1.SetOrigin(image.GetOrigin())
            image1.SetDirection(image.GetDirection())
            image = image1
            #in case of equality we do nothing
    image=subsample(image,origSpac, tuple(spacGoal))
    return image






In [12]:
path ='/home/sliceruser/data/orig/10019/10019_1000019_t2w.mha'
imageOrig = sitk.ReadImage(path)
orig_spac = imageOrig.GetSpacing()
print(orig_spac)
imageOrig.GetSize()

(0.5, 0.5, 2.999999996213984)


(384, 384, 19)

In [5]:
spacc=(0.5,0.5,1)
imagee=transformSpacingTo(path,spacc,finalSize)
imagee.GetSpacing()

1/1 [==============================] - 32s 32s/step
data shape (57, 384, 384)
new_size (384, 384, 170)
subsampled (384, 384, 170)


(0.5, 0.5, 1.0)

In [6]:
# data = sitk.GetArrayFromImage(imagee)
# data.shape

In [7]:
testImagePath = '/home/sliceruser/data/exploration/resampleddB.nii.gz'

writer = sitk.ImageFileWriter()
imagee.GetSpacing()
writer.KeepOriginalImageUIDOn()
writer.SetFileName(testImagePath)
writer.Execute(imagee)  

testImagePathB = '/home/sliceruser/data/exploration/resampledd_itk.nii.gz'
imageITKk=subsample(imageOrig, orig_spac,spacc)

writer = sitk.ImageFileWriter()
writer.KeepOriginalImageUIDOn()
writer.SetFileName(testImagePathB)
writer.Execute(imageITKk)  


new_size (384, 384, 56)
subsampled (384, 384, 56)


In [8]:
# itk_image=imageOrig
# out_spacing=(0.4,0.4,4.0)
# resample = sitk.ResampleImageFilter()
# resample.SetOutputSpacing(out_spacing)
# resample.SetSize(imageOrig.GetSize())
# resample.SetOutputDirection(itk_image.GetDirection())
# resample.SetOutputOrigin(itk_image.GetOrigin())
# resample.SetTransform(sitk.Transform())
# resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
# resample.SetInterpolator(sitk.sitkBSpline)
# resample.Execute(itk_image).GetSize()

In [9]:
imagee.GetSize()

(384, 384, 170)